In [476]:
import numpy as np
import pandas as pd

In [477]:
GEO = "GEODGHR4"
GEN_015 = "GEN_015"
SDC_015 = "SDC_015"
NDVI = "NDVI"

In [478]:
data = pd.read_csv("C:\\Users\\xia_t\\Desktop\\Projects\\youreka\\dataset\\all_subsetted_group.csv")

# filter mental health and indigenous status
data = data.loc[data[SDC_015] <= 2]
data = data.loc[data[GEN_015] <= 5]
data = data.loc[data["GEO_PRV"].isin([10, 11, 12, 13, 24, 35])]

In [479]:
# normalize NDVI into a range of [0, 1]
n = data[NDVI]
data[NDVI] = (n - n.min()) / (n.max() - n.min())

In [480]:
# categorize NDVI into N_BINS intervals
N_BINS = 4
bins = np.linspace(0, 1, N_BINS + 1)

for i in range(N_BINS):
    lower = bins[i]
    upper = bins[i + 1]

    inclusive = "right"
    if i == 0:
        inclusive = "both"

    data.loc[data["NDVI"].between(lower, upper, inclusive), "NDVI_BINS"] = i

data["NDVI"] = data["NDVI_BINS"]

In [481]:
from scipy.stats.contingency import crosstab
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

In [482]:
# chi-squared test for SDC_015
grouped = data.groupby([SDC_015])
analysis_data = grouped.apply(lambda x: x.sample(67).reset_index(drop=True))

contingency = crosstab(analysis_data[SDC_015], analysis_data[GEN_015])
contingency_table = contingency.count

print(chi2_contingency(contingency_table, correction=True))
print(association(contingency_table, method="cramer"))

Chi2ContingencyResult(statistic=6.2837995337995345, pvalue=0.1789327569130993, dof=4, expected_freq=array([[19.5, 24. , 16.5,  5.5,  1.5],
       [19.5, 24. , 16.5,  5.5,  1.5]]))
0.21655028601144427


In [483]:
# chi-squared test for NDVI
grouped = data.groupby([SDC_015])
analysis_data = grouped.apply(lambda x: x.sample(49).reset_index(drop=True))

contingency = crosstab(analysis_data[NDVI], analysis_data[GEN_015])
contingency_table = contingency.count

print(chi2_contingency(contingency_table, correction=True))
print(association(contingency_table, method="cramer"))

Chi2ContingencyResult(statistic=11.276582313837215, pvalue=0.5053760745048572, dof=12, expected_freq=array([[ 4.40816327,  6.24489796,  5.14285714,  1.83673469,  0.36734694],
       [ 9.55102041, 13.53061224, 11.14285714,  3.97959184,  0.79591837],
       [ 6.6122449 ,  9.36734694,  7.71428571,  2.75510204,  0.55102041],
       [ 3.42857143,  4.85714286,  4.        ,  1.42857143,  0.28571429]]))
0.19584616962276516


In [484]:
# chi-squared test for SDC_015 and NDVI interaction
from scipy.stats.contingency import crosstab

contingency = crosstab(analysis_data[SDC_015], analysis_data[NDVI], analysis_data[GEN_015])
contingency_table = contingency.count

from scipy.stats import chi2_contingency
chi2_contingency(contingency_table, correction=True)

Chi2ContingencyResult(statistic=27.75110887659907, pvalue=0.6339832495858966, dof=31, expected_freq=array([[[2.20408163, 3.12244898, 2.57142857, 0.91836735, 0.18367347],
        [4.7755102 , 6.76530612, 5.57142857, 1.98979592, 0.39795918],
        [3.30612245, 4.68367347, 3.85714286, 1.37755102, 0.2755102 ],
        [1.71428571, 2.42857143, 2.        , 0.71428571, 0.14285714]],

       [[2.20408163, 3.12244898, 2.57142857, 0.91836735, 0.18367347],
        [4.7755102 , 6.76530612, 5.57142857, 1.98979592, 0.39795918],
        [3.30612245, 4.68367347, 3.85714286, 1.37755102, 0.2755102 ],
        [1.71428571, 2.42857143, 2.        , 0.71428571, 0.14285714]]]))